# Race Dashboard



This document describes the requirements and design decisions we will adopt for the Race Dashboard for the Sanofi Asset Efficiency challenge.  This document is intended to provide a detailed explanation of what is to be included on the dashboard, required data sources, and assumptions being made in the design.  It will also show the steps for getting the data into the necessary format.

## Scope/objective

The objective is to be able to display the documented metrics and categories as best suited on a Dashboard presentation for Sanofi staff to be able to access.


## Metrics / categories
The metrics have been mapped into sectors to mimic different sectors of a race track.  The metrics are:
 
 
 
•     Race = 8 Laps = 8 Months   
•     Lap = Monthly Progress   
<br>
<br>

•     **Sector 1 = OEE Improvement   
•     Sector 2 = OEE Variability Improvement   
•     Sector 3 = Stoppage Reduction   
•     Sector 4 = Changeover Improvement**   
• _Sector 5 = Most effective OEE application   
• Sector 6 = Best Innovation   
• Sector 7 = Most consistent OEE improvement progress   
• Sector 8 = Collaboration   
• Sector 9 = Team Spirit_

 
 


| Change log |
|:----------:|    


| Date | Initials | Comments |
|------|:---------|:---------|
| 2021-06-23 | MC | in leaderboard, replace NaN values in laptime calc with the max laptime for that lap
| 2021-06-23 | MC | use race review dates for grouping data, rather than calendar months
| 2021-06-24 |JB | missing OEE_Diff figures should default to 'OEE %' - OEE start point, not just 'OEE %' value
| 2021-06-24 |JB | for sector one, multiply sum of OEE_Diff by -1.  Should have always been doing this.
|2021-06-25 | MC | in leaderboard, change prev_race_time calc to include all but last 2 cols, to handle new race cols as they arrive

In [1]:
import pandas as pd
import numpy as np
import datetime


# Viz libs
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# display options
# pd.options.display.float_format = "{:.2f}".format


## Read file and cleanse

- QSWeeklyFigures.xlsx should contain the latest data from QS   
- QSDashboard.xlsx has a list of plants/sites taking part, with original target OEE  
- QSWeeklyUnplannedTechLoss.xles should contain the latest data from QS (my unplanned chart by line/week)

#### Cleaning required:   
- OEE % needs converting to numeric, coerce the nulls to nan values
- I think W53-2020 data is bobbings - causes dup indexes for date 2021-01-10 and don't need 2020 data anyway...so dropping it

In [2]:
dir = "C:/Users/mark_/McLaren Technology Group/McLaren Accelerator - Sanofi - Sanofi/Data Analysis/"
output_dir = "C:/Users/mark_/Documents/McLaren2021/Sanofi/"

# dir = 'C:/Users/james.blood/Documents/McLarenSanofi/McLarenSanofi/data/'

file = (dir + 'OEE.xlsx')
df_weekly = pd.read_excel(file)
file = (dir + 'QSDashboard.xlsx')
df_dash = pd.read_excel(file)
file = (dir + 'Unplanned_tech_loss.xlsx')
df_techloss = pd.read_excel(file)
file = (dir + 'changeover.xlsx')
df_changeover = pd.read_excel(file)
df_weekly = df_weekly.loc[df_weekly['Week'].str.contains('2021')]
df_techloss = df_techloss.loc[df_techloss['Week'].str.contains('2021')]
df_weekly['OEE %'] = pd.to_numeric(df_weekly['OEE %'], errors='coerce')
df_techloss.rename(columns={'Unplanned losses - %OEE':'Unplanned_tech_loss'}, inplace=True)
df_techloss['Unplanned_tech_loss'] = pd.to_numeric(df_techloss['Unplanned_tech_loss'], errors='coerce')
df_changeover.rename(columns={'Change over losses - %OEE':'Changeover'}, inplace=True)
df_changeover['Changeover'] = pd.to_numeric(df_changeover['Changeover'], errors='coerce')
# don't use their progress figure as it's a static val
# df_dash.rename(columns={'⇗ OEE% progress':'OEE% progress'}, inplace=True)

create a datetime from the week number

In [3]:
df_weekly['WeekOfYear'] = pd.to_numeric(df_weekly['Week'].str[1:3])
df_weekly['Year'] = pd.to_numeric(df_weekly['Week'].str[4:])
dates = df_weekly.Year*100+df_weekly.WeekOfYear
df_weekly['Date'] = pd.to_datetime(dates.astype(str) + '0', format='%Y%W%w')
# df_weekly.drop(columns=['Year','WeekOfYear'], inplace=True)
df_weekly.head()

,Week,Line,OEE %,WeekOfYear,Year,Date
11,W01-2021,C2 Packaging Line,0.168970,1,2021,2021-01-10
12,W01-2021,C9 Packaging Line,NaN,1,2021,2021-01-10
13,W01-2021,GAMMA1,0.406686,1,2021,2021-01-10
14,W01-2021,IMA C80/2,0.510044,1,2021,2021-01-10
15,W01-2021,L18 Packaging Line,0.173736,1,2021,2021-01-10


In [4]:
#merge the 2 dataframes to get the start OEE
df_weekly = df_weekly.merge(df_dash[['Plant','Line', 'OEE  Start point','OEE% Target (2022)']],on='Line')

In [5]:
df_weekly = df_weekly.merge(df_techloss[['Line', 'Week', 'Unplanned_tech_loss']],on=['Line','Week'])

In [6]:
df_weekly = df_weekly.merge(df_changeover[['Line','Week','Changeover']])

#### Start Changeover

Start changeover value isn't provided, so going to calc our own start point using the average changeover for each site in 2021 up to April 2021.  This needs to be done before we drop the early 2021 rows.

This is then merged into the df_weekly dataframe as a loose join.

In [7]:
start_changeover_calc = df_weekly[['Plant','Line','Changeover']][df_weekly['Date'] < '2021-04-30'].groupby(['Plant','Line']).mean().reset_index()
start_changeover_calc.rename(columns={'Changeover':'start_changeover'}, inplace=True)
df_weekly = df_weekly.merge(start_changeover_calc[['Line','start_changeover']])

Turn decimals into percentages before we go any calcs?? Not doing this at the moment as it's useful having similar values for calculating the sector times later.

In [8]:
# df_weekly[['OEE %','OEE  Start point','OEE% Target (2022)','Unplanned_tech_loss','Changeover']] = df_weekly[['OEE %','OEE  Start point','OEE% Target (2022)','Unplanned_tech_loss','Changeover']] * 100

#### Dates for the Asset Challenge

Start Date is going to be fixed as 2021-04-01. Remove all the rows from df_weekly before this date

End Date will move and act as a cutoff before each Race meeting

In [9]:
start_date = '2021-04-01'
df_weekly = df_weekly[df_weekly['Date'] > start_date].sort_values('Date')

# do we need this?  We now have race review dates
end_date = '2021-07-15'
df_weekly = df_weekly[df_weekly['Date'] < end_date].sort_values('Date')

### PCT_CHANGE
Using pct_change python function with periods=4, giving a 4 week (4 previous rows) rolling pct_change figure

- Not sure whether this is required for all of the categories?

In [10]:
df_weekly.sort_values(['Line','Date'], inplace = True)
df_weekly['OEE_pct_chg'] = (df_weekly.groupby('Line')['OEE %']
                                   .apply(pd.Series.pct_change, periods=4))
df_weekly['techloss_pct_chg'] = (df_weekly.groupby('Line')['Unplanned_tech_loss']
                                   .apply(pd.Series.pct_change, periods=4))
df_weekly['Changeover_pct_chg'] = (df_weekly.groupby('Line')['Changeover']
                                   .apply(pd.Series.pct_change, periods=4))
df_weekly.head()

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover,start_changeover,OEE_pct_chg,techloss_pct_chg,Changeover_pct_chg
253,W13-2021,AL5 Packaging 1,NaN,13,2021,2021-04-04,Frankfurt,0.479693,0.5,NaN,NaN,0.104883,NaN,NaN,NaN
254,W14-2021,AL5 Packaging 1,NaN,14,2021,2021-04-11,Frankfurt,0.479693,0.5,NaN,NaN,0.104883,NaN,NaN,NaN
255,W15-2021,AL5 Packaging 1,0.449745,15,2021,2021-04-18,Frankfurt,0.479693,0.5,0.314964,0.087039,0.104883,NaN,NaN,NaN
256,W16-2021,AL5 Packaging 1,0.642652,16,2021,2021-04-25,Frankfurt,0.479693,0.5,0.135051,0.122727,0.104883,NaN,NaN,NaN
257,W17-2021,AL5 Packaging 1,0.505804,17,2021,2021-05-02,Frankfurt,0.479693,0.5,0.280638,0.063261,0.104883,NaN,NaN,NaN


## Standard Deviation
Calculate std_dev and mean on a 4 week rolling basis

Standard deviation is the square root of the variance, so no need to calculate both and have left var out

In [11]:
df_weekly['rolling_std'] = df_weekly.groupby('Line')['OEE %'].apply(lambda x : x.rolling(4,1).agg(np.std))
df_weekly.head(50)

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover,start_changeover,OEE_pct_chg,techloss_pct_chg,Changeover_pct_chg,rolling_std
253,W13-2021,AL5 Packaging 1,NaN,13,2021,2021-04-04,Frankfurt,0.479693,0.50,NaN,NaN,0.104883,NaN,NaN,NaN,NaN
254,W14-2021,AL5 Packaging 1,NaN,14,2021,2021-04-11,Frankfurt,0.479693,0.50,NaN,NaN,0.104883,NaN,NaN,NaN,NaN
255,W15-2021,AL5 Packaging 1,0.449745,15,2021,2021-04-18,Frankfurt,0.479693,0.50,0.314964,0.087039,0.104883,NaN,NaN,NaN,NaN
256,W16-2021,AL5 Packaging 1,0.642652,16,2021,2021-04-25,Frankfurt,0.479693,0.50,0.135051,0.122727,0.104883,NaN,NaN,NaN,0.136405
257,W17-2021,AL5 Packaging 1,0.505804,17,2021,2021-05-02,Frankfurt,0.479693,0.50,0.280638,0.063261,0.104883,NaN,NaN,NaN,0.099233
258,W18-2021,AL5 Packaging 1,0.443611,18,2021,2021-05-09,Frankfurt,0.479693,0.50,0.302882,0.089293,0.104883,NaN,NaN,NaN,0.092469
259,W19-2021,AL5 Packaging 1,0.507565,19,2021,2021-05-16,Frankfurt,0.479693,0.50,0.230673,0.060573,0.104883,0.128560,-0.267622,-0.304075,0.083941
260,W20-2021,AL5 Packaging 1,NaN,20,2021,2021-05-23,Frankfurt,0.479693,0.50,NaN,NaN,0.104883,-0.210202,0.708047,-0.506445,0.036426
261,W21-2021,AL5 Packaging 1,NaN,21,2021,2021-05-30,Frankfurt,0.479693,0.50,NaN,NaN,0.104883,0.003481,-0.178041,-0.042495,0.045222
262,W22-2021,AL5 Packaging 1,0.000000,22,2021,2021-06-06,Frankfurt,0.479693,0.50,0.000000,0.080000,0.104883,-1.000000,-1.000000,-0.104077,0.358903


### Calculating Sector times


New instructions 2021-06-11:

The lap time is defined by the sum of the sector scores, with the lowest total equalling the pole time for Ricard.  The following times will be pro-rated against this time (for the total sector score recorded) to give a slightly longer overall lap time.    



Sector [1-4] calculations   
**sector 1**
How much has your OEE increased / decreased?  Sum difference between each week

df_weekly['sector_1'] = df_weekly['OEE_Diff']

OEE_Diff calculation
- Sort values by Line and Date
- Find the difference between each weekly OEE figure
- Fill NaN values from missing OEE figures with the weekly OEE minus OEE Start Point for that site


**Sector 2** 
How big was your rolling std deviation this period, over the previous 4 weeks std dev?

df_weekly['sector_2'] = df_weekly['rolling_std']

rolling_std = rolling std deviation for past 4 weeks for each site


 
**Sector 3**
We want to reduce Unplanned tech loss (recorded as % of OEE) Unplanned tech loss is calculated within QlikSense but missing values sometimes.  Fill the missing values and then display the average Unplanned tech loss :

df_weekly['sector_3'] = df_weekly['Unplanned_tech_loss']


Populate missing unplanned tech loss:
- Create weekly min/max cols for Unplanned tech loss from any site 
- Merge those columns into df_weekly 
- fill any NaN unplanned tech loss rows with the max OEE calc'd for that week (bigger is worse)

 
**Sector 4**
We're trying to reduce changeover time (recorded as % of OEE).  
Start changeover value isn't provided, so calc our own start point for each Line using the average changeover in 2021 up to 30 April, 2021.
   
start_changeover_calc = df_weekly[['Plant','Line','Changeover']][df_weekly['Date'] < '2021-04-30'].groupby(['Plant','Line']).mean().reset_index()
start_changeover_calc.rename(columns={'Changeover':'start_changeover'}, inplace=True)
df_weekly = df_weekly.merge(start_changeover_calc[['Line','start_changeover']])



df_weekly['sector_4'] = df_weekly['Changeover_rolling_mean']

Changeover_mean = df_weekly.sort_values(by=['Line', 'Date'])[['Line', 'Date', 'Changeover', 'start_changeover']]
Changeover_mean['Changeover_rolling_mean'] = Changeover_mean.groupby('Line')['Changeover'].apply(lambda x : x.rolling(4,1).mean())
df_weekly = df_weekly.merge(Changeover_mean[["Line","Date","Changeover_rolling_mean"]], on=(["Line","Date"]))



**Clean the sectors of NaN before summing them**   
Sometimes, when we haven't got enough information for pct_change calcs, we were getting no values coming through for the lap_time.  We should make sure there is a value in each of the sectors, otherwise there is an unfair advantage by not having data available.  Find all NaN values and replace with the mean for that column(sector)

**Sectors 5 - 9**   
These scores are taken from the Nomination process.  Read in the Nomination s/s, merge any values we find with df_weekly, replace all NaN (missing) values with 0, and reduce the scores we find to 10% of their original value.  This value is then subtracted from the lap_time - so the better you do in the nominations the more your lap_time gets reduced by.

 
**lap_time**
df_weekly['lap_time'] = df_weekly[['sector_1','sector_2','sector_3','sector_4']].sum(axis=1)

In [12]:
file = (dir + 'Nominations Category Scoring.xlsx')
df_nom_sectors = pd.read_excel(file, sheet_name='Nomination scoring', usecols="A:H", parse_dates=['Date'])

In [13]:
df_nom_sectors['Date'] = pd.Series(df_nom_sectors['Date']).fillna(method='ffill')
df_nom_sectors = df_nom_sectors.fillna(0)

df_weekly = df_weekly.merge(df_nom_sectors[['Line','Plant','Date','Best Solution','Best Innovation','Improvement Iterations','Lessons and Sharing','Team Contribution and Spirit']], how='outer', on=['Date','Plant','Line'])
df_weekly

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,...,start_changeover,OEE_pct_chg,techloss_pct_chg,Changeover_pct_chg,rolling_std,Best Solution,Best Innovation,Improvement Iterations,Lessons and Sharing,Team Contribution and Spirit
0,W13-2021,AL5 Packaging 1,NaN,13.0,2021.0,2021-04-04,Frankfurt,0.479693,0.5,NaN,...,0.104883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,W14-2021,AL5 Packaging 1,NaN,14.0,2021.0,2021-04-11,Frankfurt,0.479693,0.5,NaN,...,0.104883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,W15-2021,AL5 Packaging 1,0.449745,15.0,2021.0,2021-04-18,Frankfurt,0.479693,0.5,0.314964,...,0.104883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,W16-2021,AL5 Packaging 1,0.642652,16.0,2021.0,2021-04-25,Frankfurt,0.479693,0.5,0.135051,...,0.104883,NaN,NaN,NaN,0.136405,NaN,NaN,NaN,NaN,NaN
4,W17-2021,AL5 Packaging 1,0.505804,17.0,2021.0,2021-05-02,Frankfurt,0.479693,0.5,0.280638,...,0.104883,NaN,NaN,NaN,0.099233,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,NaN,AL5 Packaging 1,NaN,NaN,NaN,2021-09-30,Frankfurt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
202,NaN,AL6,NaN,NaN,NaN,2021-09-30,Frankfurt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
203,NaN,M18 Filling,NaN,NaN,NaN,2021-09-30,Frankfurt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
204,NaN,M21 Filling,NaN,NaN,NaN,2021-09-30,Frankfurt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


#### Create review dates
create a review_date column for grouping the data later, so we only get the data we're interested in for each review

In [14]:
def thurs_of_weekbefore(year, week):
    return datetime.date.fromisocalendar(year, week-1, 4)  # (year, week before (w-1), thursday)

review_weeks = [16, 20, 24, 28, 34, 38, 42, 47]
review_dates = []

for i in review_weeks:
    if i > 0:
        review_dates.append((thurs_of_weekbefore(2021,i)))

df_review_dates = pd.DataFrame(review_dates)
df_review_dates.rename(columns={0:'Review_Date'}, inplace=True)
df_review_dates['Review_Date'] = pd.to_datetime(df_review_dates.Review_Date)

# df_review_dates.info()
df_weekly = pd.merge_asof(df_weekly.sort_values('Date'), df_review_dates, left_on='Date', right_on='Review_Date', direction='forward')


#### Populate missing OEE %
- Find the weekly min/max OEE % from any site   
- Merge those columns into df_weekly   
- fill any NaN with the min OEE we calc'd for that week   

In [15]:
df_weekly_minmax = (df_weekly.assign(Data_Value=df_weekly['OEE %'].abs())
       .groupby(pd.Grouper(key='Date',freq='W'))['OEE %'].agg([('Min' , 'min'), ('Max', 'max')])
       .add_prefix('Week'))
df_weekly_minmax.reset_index(inplace=True)
df_weekly = df_weekly.merge(df_weekly_minmax[['Date','WeekMin','WeekMax']])
df_weekly['OEE %'].fillna(df_weekly.WeekMin, inplace=True)
df_weekly

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,...,Changeover_pct_chg,rolling_std,Best Solution,Best Innovation,Improvement Iterations,Lessons and Sharing,Team Contribution and Spirit,Review_Date,WeekMin,WeekMax
0,W13-2021,AL5 Packaging 1,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.479693,0.500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.046124,0.649336
1,W13-2021,C2 Packaging Line,0.458414,13.0,2021.0,2021-04-04,Maisons-Alfort,0.397503,0.470,0.465009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.046124,0.649336
2,W13-2021,M18 Filling,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.443522,0.650,0.114818,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.046124,0.649336
3,W13-2021,AL6,0.367897,13.0,2021.0,2021-04-04,Frankfurt,0.332657,0.450,0.342851,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.046124,0.649336
4,W13-2021,SUPPO Packaging Line,0.432432,13.0,2021.0,2021-04-04,Lisieux,0.353021,0.530,0.148267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.046124,0.649336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,W23-2021,IMA C80/2,0.515246,23.0,2021.0,2021-06-13,SCOPPITO,0.451031,0.580,0.188204,...,0.452974,0.051457,0.0,67.0,0.0,0.0,78.0,2021-07-08,0.282243,0.677166
142,W23-2021,SUPPO Packaging Line,0.282243,23.0,2021.0,2021-06-13,Lisieux,0.353021,0.530,NaN,...,inf,0.077611,0.0,0.0,0.0,0.0,0.0,2021-07-08,0.282243,0.677166
143,W23-2021,C2 Packaging Line,0.467328,23.0,2021.0,2021-06-13,Maisons-Alfort,0.397503,0.470,0.434789,...,inf,0.051397,0.0,0.0,0.0,0.0,0.0,2021-07-08,0.282243,0.677166
144,W23-2021,L25 Packaging Line,0.318505,23.0,2021.0,2021-06-13,Tours,0.351564,0.478,0.214649,...,-0.278937,0.033693,0.0,0.0,0.0,0.0,0.0,2021-07-08,0.282243,0.677166


#### We need the diff between the weekly OEE % figures, and the Weekly Changeover figures
Need something to calculate the OEE Progress and Changeover, otherwise we will have problems when we group and sum values later
- Create OEE_diff with OEE % from groupby of each Line, Week (only 1 row per week, so 'mean' will yield the same)   
- Find the diff between the rows in OEE_Diff for each Line   
- fillNA (first row for each Line) with OEE Start point - should only be needed on the first row for each Line   

repeat same logic for Changeover - there will be more NaN as start_changeover wasn't provided for all.  We populate this later

In [16]:
# this was calculating the wrong Diff - the first row of each site was looking at the previous site for all but the 1st calc
# needed to sort by Line and Date first 

# OEE_Diff = df_weekly.groupby(['Line',pd.Grouper(key='Date',freq='W')])['OEE %'].mean().reset_index()
# OEE_Diff["OEE_Diff"] = OEE_Diff["OEE %"].diff()
# df_weekly = df_weekly.merge(OEE_Diff[["Line","Date","OEE_Diff"]], on=(["Line","Date"]))

# df_weekly['OEE_Diff'].fillna(df_weekly['OEE %'] - df_weekly['OEE  Start point'], inplace=True)
# df_weekly[["Line","Date","OEE %","OEE_Diff"]].head(50).sort_values(by=['Line', 'Date'])

In [17]:
df_weekly[df_weekly.Line.str.contains('AL5')]

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,...,Changeover_pct_chg,rolling_std,Best Solution,Best Innovation,Improvement Iterations,Lessons and Sharing,Team Contribution and Spirit,Review_Date,WeekMin,WeekMax
0,W13-2021,AL5 Packaging 1,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.479693,0.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.046124,0.649336
20,W14-2021,AL5 Packaging 1,0.346988,14.0,2021.0,2021-04-11,Frankfurt,0.479693,0.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.346988,0.530707
27,W15-2021,AL5 Packaging 1,0.449745,15.0,2021.0,2021-04-18,Frankfurt,0.479693,0.5,0.314964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-13,0.187845,0.655470
49,W16-2021,AL5 Packaging 1,0.642652,16.0,2021.0,2021-04-25,Frankfurt,0.479693,0.5,0.135051,...,NaN,0.136405,NaN,NaN,NaN,NaN,NaN,2021-05-13,0.000000,0.703703
62,W17-2021,AL5 Packaging 1,0.505804,17.0,2021.0,2021-05-02,Frankfurt,0.479693,0.5,0.280638,...,NaN,0.099233,NaN,NaN,NaN,NaN,NaN,2021-05-13,0.352339,0.707643
69,W18-2021,AL5 Packaging 1,0.443611,18.0,2021.0,2021-05-09,Frankfurt,0.479693,0.5,0.302882,...,NaN,0.092469,NaN,NaN,NaN,NaN,NaN,2021-05-13,0.356017,0.769172
81,W19-2021,AL5 Packaging 1,0.507565,19.0,2021.0,2021-05-16,Frankfurt,0.479693,0.5,0.230673,...,-0.304075,0.083941,NaN,NaN,NaN,NaN,NaN,2021-06-10,0.000000,0.733994
98,W20-2021,AL5 Packaging 1,0.000000,20.0,2021.0,2021-05-23,Frankfurt,0.479693,0.5,NaN,...,-0.506445,0.036426,NaN,NaN,NaN,NaN,NaN,2021-06-10,0.000000,0.698169
109,W21-2021,AL5 Packaging 1,0.191951,21.0,2021.0,2021-05-30,Frankfurt,0.479693,0.5,NaN,...,-0.042495,0.045222,NaN,NaN,NaN,NaN,NaN,2021-06-10,0.191951,0.706192
126,W22-2021,AL5 Packaging 1,0.000000,22.0,2021.0,2021-06-06,Frankfurt,0.479693,0.5,0.000000,...,-0.104077,0.358903,NaN,NaN,NaN,NaN,NaN,2021-06-10,0.000000,0.769486


In [18]:
OEE_Diff = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','OEE %','OEE  Start point']]
OEE_Diff['OEE_Diff'] = OEE_Diff.groupby('Line')['OEE %'].diff().fillna(df_weekly['OEE %'] - df_weekly['OEE  Start point'])
df_weekly = df_weekly.merge(OEE_Diff[["Line","Date","OEE_Diff"]], on=(["Line","Date"]))
df_weekly[["Line","Date","OEE %","OEE_Diff"]].head(50).sort_values(by=['Line', 'Date'])

,Line,Date,OEE %,OEE_Diff
0,AL5 Packaging 1,2021-04-04,0.046124,-0.433569
20,AL5 Packaging 1,2021-04-11,0.346988,0.300865
27,AL5 Packaging 1,2021-04-18,0.449745,0.102757
49,AL5 Packaging 1,2021-04-25,0.642652,0.192906
3,AL6,2021-04-04,0.367897,0.035240
24,AL6,2021-04-11,0.360681,-0.007216
34,AL6,2021-04-18,0.334140,-0.026541
1,C2 Packaging Line,2021-04-04,0.458414,0.060911
19,C2 Packaging Line,2021-04-11,0.530707,0.072293
36,C2 Packaging Line,2021-04-18,0.410676,-0.120032


In [19]:
# Changeover_Diff = df_weekly.groupby(['Line',pd.Grouper(key='Date',freq='W')])['Changeover'].mean().reset_index()
# Changeover_Diff["Changeover_Diff"] = Changeover_diff["Changeover"].diff()
# df_weekly = df_weekly.merge(Changeover_diff[["Line","Date","Changeover_Diff"]], on=(["Line","Date"]))

# df_weekly['Changeover_Diff'].fillna(df_weekly['start_changeover'] - df_weekly['Changeover'], inplace=True)

In [20]:
Changeover_Diff = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','Changeover','start_changeover']]
Changeover_Diff['Changeover_Diff'] = Changeover_Diff.groupby('Line')['Changeover'].diff().fillna(df_weekly['start_changeover'] - df_weekly['Changeover'])
df_weekly = df_weekly.merge(Changeover_Diff[["Line","Date","Changeover_Diff"]], on=(["Line","Date"]))

In [21]:
Changeover_mean = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','Changeover','start_changeover']]
Changeover_mean['Changeover_rolling_mean'] = Changeover_mean.groupby('Line')['Changeover'].apply(lambda x : x.rolling(4,1).mean())
df_weekly = df_weekly.merge(Changeover_mean[["Line","Date","Changeover_rolling_mean"]], on=(["Line","Date"]))

#### Populate missing Unplanned Tech Loss

- Create weekly min/max cols for Unplanned tech loss from any site   
- Merge those columns into df_weekly   
- fill any NaN rows with the max OEE calc'd for that week   

**this might be flawed!!** 

In [22]:
df_weekly_minmax = (df_weekly.assign(Data_Value=df_weekly['Unplanned_tech_loss'].abs())
       .groupby(pd.Grouper(key='Date',freq='W'))['Unplanned_tech_loss'].agg([('Min' , 'min'), ('Max', 'max')])
       .add_prefix('WeekUTL'))
df_weekly_minmax.reset_index(inplace=True)
df_weekly = df_weekly.merge(df_weekly_minmax[['Date','WeekUTLMin','WeekUTLMax']])
df_weekly['Unplanned_tech_loss'].fillna(df_weekly.WeekUTLMax, inplace=True)
df_weekly

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,...,Lessons and Sharing,Team Contribution and Spirit,Review_Date,WeekMin,WeekMax,OEE_Diff,Changeover_Diff,Changeover_rolling_mean,WeekUTLMin,WeekUTLMax
0,W13-2021,AL5 Packaging 1,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.479693,0.500,0.465009,...,NaN,NaN,2021-04-15,0.046124,0.649336,-0.433569,NaN,NaN,0.114818,0.465009
1,W13-2021,C2 Packaging Line,0.458414,13.0,2021.0,2021-04-04,Maisons-Alfort,0.397503,0.470,0.465009,...,NaN,NaN,2021-04-15,0.046124,0.649336,0.060911,0.007567,0.044140,0.114818,0.465009
2,W13-2021,M18 Filling,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.443522,0.650,0.114818,...,NaN,NaN,2021-04-15,0.046124,0.649336,-0.397398,-0.542447,0.789781,0.114818,0.465009
3,W13-2021,AL6,0.367897,13.0,2021.0,2021-04-04,Frankfurt,0.332657,0.450,0.342851,...,NaN,NaN,2021-04-15,0.046124,0.649336,0.035240,-0.077980,0.184958,0.114818,0.465009
4,W13-2021,SUPPO Packaging Line,0.432432,13.0,2021.0,2021-04-04,Lisieux,0.353021,0.530,0.148267,...,NaN,NaN,2021-04-15,0.046124,0.649336,0.079411,-0.077059,0.178806,0.114818,0.465009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,W23-2021,IMA C80/2,0.515246,23.0,2021.0,2021-06-13,SCOPPITO,0.451031,0.580,0.188204,...,0.0,78.0,2021-07-08,0.282243,0.677166,0.010669,0.067577,0.191808,0.102172,0.434789
142,W23-2021,SUPPO Packaging Line,0.282243,23.0,2021.0,2021-06-13,Lisieux,0.353021,0.530,0.434789,...,0.0,0.0,2021-07-08,0.282243,0.677166,-0.097105,NaN,0.050597,0.102172,0.434789
143,W23-2021,C2 Packaging Line,0.467328,23.0,2021.0,2021-06-13,Maisons-Alfort,0.397503,0.470,0.434789,...,0.0,0.0,2021-07-08,0.282243,0.677166,0.039284,-0.061726,0.054718,0.102172,0.434789
144,W23-2021,L25 Packaging Line,0.318505,23.0,2021.0,2021-06-13,Tours,0.351564,0.478,0.214649,...,0.0,0.0,2021-07-08,0.282243,0.677166,-0.037553,-0.028535,0.238142,0.102172,0.434789


#### Populate missing Changeover 

In [23]:
df_weekly_minmax = (df_weekly.assign(Data_Value=df_weekly['Changeover'].abs())
       .groupby(pd.Grouper(key='Date',freq='W'))['Changeover'].agg([('Min' , 'min'), ('Max', 'max')])
       .add_prefix('WeekChangeover'))
df_weekly_minmax.reset_index(inplace=True)
df_weekly = df_weekly.merge(df_weekly_minmax[['Date','WeekChangeoverMin','WeekChangeoverMax']])
df_weekly['Changeover'].fillna(df_weekly.WeekChangeoverMax, inplace=True)
df_weekly

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,...,Review_Date,WeekMin,WeekMax,OEE_Diff,Changeover_Diff,Changeover_rolling_mean,WeekUTLMin,WeekUTLMax,WeekChangeoverMin,WeekChangeoverMax
0,W13-2021,AL5 Packaging 1,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.479693,0.500,0.465009,...,2021-04-15,0.046124,0.649336,-0.433569,NaN,NaN,0.114818,0.465009,0.035164,0.789781
1,W13-2021,C2 Packaging Line,0.458414,13.0,2021.0,2021-04-04,Maisons-Alfort,0.397503,0.470,0.465009,...,2021-04-15,0.046124,0.649336,0.060911,0.007567,0.044140,0.114818,0.465009,0.035164,0.789781
2,W13-2021,M18 Filling,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.443522,0.650,0.114818,...,2021-04-15,0.046124,0.649336,-0.397398,-0.542447,0.789781,0.114818,0.465009,0.035164,0.789781
3,W13-2021,AL6,0.367897,13.0,2021.0,2021-04-04,Frankfurt,0.332657,0.450,0.342851,...,2021-04-15,0.046124,0.649336,0.035240,-0.077980,0.184958,0.114818,0.465009,0.035164,0.789781
4,W13-2021,SUPPO Packaging Line,0.432432,13.0,2021.0,2021-04-04,Lisieux,0.353021,0.530,0.148267,...,2021-04-15,0.046124,0.649336,0.079411,-0.077059,0.178806,0.114818,0.465009,0.035164,0.789781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,W23-2021,IMA C80/2,0.515246,23.0,2021.0,2021-06-13,SCOPPITO,0.451031,0.580,0.188204,...,2021-07-08,0.282243,0.677166,0.010669,0.067577,0.191808,0.102172,0.434789,0.000000,0.280400
142,W23-2021,SUPPO Packaging Line,0.282243,23.0,2021.0,2021-06-13,Lisieux,0.353021,0.530,0.434789,...,2021-07-08,0.282243,0.677166,-0.097105,NaN,0.050597,0.102172,0.434789,0.000000,0.280400
143,W23-2021,C2 Packaging Line,0.467328,23.0,2021.0,2021-06-13,Maisons-Alfort,0.397503,0.470,0.434789,...,2021-07-08,0.282243,0.677166,0.039284,-0.061726,0.054718,0.102172,0.434789,0.000000,0.280400
144,W23-2021,L25 Packaging Line,0.318505,23.0,2021.0,2021-06-13,Tours,0.351564,0.478,0.214649,...,2021-07-08,0.282243,0.677166,-0.037553,-0.028535,0.238142,0.102172,0.434789,0.000000,0.280400


#### Populate missing start_changeover (NaN)
If we've got this far and you still don't have a start_changeover, then you're a new site and can have this week's Changeover value

These rows still have NaN Changeover values

In [24]:
df_weekly[['Line','Date','Changeover','start_changeover']][df_weekly['Changeover'].isna()]

,Line,Date,Changeover,start_changeover


In [25]:
df_weekly.start_changeover.fillna(df_weekly.Changeover, inplace=True)
df_weekly['OEE  Start point'].fillna(df_weekly['OEE %'], inplace=True)
df_weekly['OEE% Target (2022)'].fillna(0.65, inplace=True)

#### Sector times

In [26]:
# df_weekly['sector_1'] = (df_weekly['WeekMax'] - df_weekly['OEE %'])
# df_weekly['sector_1'] = (df_weekly['OEE  Start point'] - df_weekly['OEE %'])
df_weekly['sector_1'] = df_weekly['OEE_Diff'].mul(-1)
df_weekly['sector_2'] = df_weekly['rolling_std']
df_weekly['sector_3'] = df_weekly['Unplanned_tech_loss']
# df_weekly['sector_4'] = (df_weekly['start_changeover'] - df_weekly['Changeover'])
# df_weekly['sector_4'] = (df_weekly['Changeover'] - df_weekly['start_changeover'])
df_weekly['sector_4'] = df_weekly['Changeover_rolling_mean']
# take 10% of the sector5-9 scores 
df_weekly[['sector_5','sector_6','sector_7','sector_8','sector_9']] = df_weekly[['Best Solution','Best Innovation','Improvement Iterations','Lessons and Sharing','Team Contribution and Spirit']] * -0.1
df_weekly[['sector_5','sector_6','sector_7','sector_8','sector_9']] = df_weekly[['sector_5','sector_6','sector_7','sector_8','sector_9']].fillna(0)

# we'll use these in the absence of values for a sector
df_weekly[['sector_1','sector_2','sector_3','sector_4']] = df_weekly[['sector_1','sector_2','sector_3','sector_4']].fillna(df_weekly[['sector_1','sector_2','sector_3','sector_4']].mean())

#this will sum and handle the NaN
df_weekly['lap_time'] = df_weekly[['sector_1','sector_2','sector_3','sector_4','sector_5','sector_6','sector_7','sector_8','sector_9']].sum(axis=1)

# now add the pole['Laptime'] from fastf1 to the lap_time adjustment we've created
# just use 88 secs rather than playing with timedeltas for now
# df_weekly['lap_time'] = pole['LapTime'] + pd.to_timedelta(df_weekly['lap_time'], unit='S')
# df_weekly['lap_time'] = 88 + df_weekly['lap_time']
df_weekly.groupby(['Line', pd.Grouper(key='Date', freq='W')])['lap_time'].sum()
# print (df_weekly['sector_1_time'] , df_weekly['sector_2_time'] , df_weekly['sector_3_time'], df_weekly['sector_4_time'])

Line                  Date      
AL5 Packaging 1       2021-04-04    1.160776
                      2021-04-11    0.355378
                      2021-04-18    0.399490
                      2021-04-25    0.183433
                      2021-05-02    0.607727
                                      ...   
TR200 Packaging Line  2021-05-16    0.392889
                      2021-05-23    0.298976
                      2021-05-30    0.527540
                      2021-06-06    0.162539
                      2021-06-13    0.704211
Name: lap_time, Length: 146, dtype: float64

#### Write out df_weekly to excel

In [27]:
df_weekly.to_excel(output_dir + "df_weekly_with_calcs.xlsx")

#### Monthly Calcs

Repeat the process for a df_monthly spreadsheet.  We will use this for calculating the Leader board  
group df_weekly by review_date so we can get the right data for each review meeting

In [28]:
# df_monthly = df_weekly.groupby([pd.Grouper(key='Date',freq='M'),'Line'])[['start_changeover','OEE  Start point','OEE %','Unplanned_tech_loss','Changeover','rolling_std','techloss_pct_chg','Changeover_pct_chg']].mean().reset_index()
# df_monthly = df_weekly.groupby([pd.Grouper(key='Date',freq='M'),'Line']).lap_time.sum().reset_index()
df_monthly = df_weekly.groupby(['Review_Date','Line']).lap_time.sum().reset_index()
# change the name of review_date to save renaming all references to Date later
df_monthly = df_monthly.rename(columns={'Review_Date':'Date'})
df_monthly

,Date,Line,lap_time
0,2021-04-15,AL5 Packaging 1,1.516153
1,2021-04-15,AL6,1.140448
2,2021-04-15,C2 Packaging Line,0.955284
3,2021-04-15,C9 Packaging Line,1.080853
4,2021-04-15,GAMMA1,1.156164
5,2021-04-15,IMA C80/2,0.912853
6,2021-04-15,L18 Packaging Line,0.849900
7,2021-04-15,L25 Packaging Line,0.852377
8,2021-04-15,M18 Filling,1.963066
9,2021-04-15,M21 Filling,1.636132


In [29]:
# df_monthly_minmax = (df_weekly.assign(Data_Value=df_weekly['OEE %'].abs())
#        .groupby(pd.Grouper(key='Date',freq='M'))['OEE %'].agg([('Min' , 'min'), ('Max', 'max')])
#        .add_prefix('Month'))
# df_monthly_minmax.reset_index(inplace=True)
# df_monthly = df_monthly.merge(df_monthly_minmax[['Date','MonthMin','MonthMax']])
df_monthly['lap_time'] = df_monthly['lap_time'] + 88
df_monthly

,Date,Line,lap_time
0,2021-04-15,AL5 Packaging 1,89.516153
1,2021-04-15,AL6,89.140448
2,2021-04-15,C2 Packaging Line,88.955284
3,2021-04-15,C9 Packaging Line,89.080853
4,2021-04-15,GAMMA1,89.156164
5,2021-04-15,IMA C80/2,88.912853
6,2021-04-15,L18 Packaging Line,88.849900
7,2021-04-15,L25 Packaging Line,88.852377
8,2021-04-15,M18 Filling,89.963066
9,2021-04-15,M21 Filling,89.636132


### Leader board table

In [36]:
# filter using the end_date to stop picking up future dated nomination rows of zero I created when joining the s/s
pivot = df_monthly[df_monthly['Date'] < end_date].pivot(index='Line', columns='Date', values='lap_time')
pivot.reset_index(inplace=True)
# pivot creates NaN for rows with no monthly data for each race review data
# populate each NaN value with the max for that column - so they get the max laptime for that race
# we can search for cols [1:] and find all cols after Date and Line
pivot.iloc[:,1:] = pivot.iloc[:,1:].fillna(pivot.iloc[:,1:].max())

# sum all the columns to get a race_time
pivot['race_time'] = pivot.sum(axis=1)
# sum all but the last 2 cols (this lap and the race_time) to calc prev_race_time

# pivot['prev_race_time'] = pivot[pivot.columns[2]] + pivot[pivot.columns[3]]
pivot['prev_race_time'] = pivot.iloc[:,1:-2].sum(axis=1)

pivot = pivot.merge(df_dash[['Plant','Line']], on='Line')
pivot.sort_values('race_time', inplace=True)
pivot['position'] = np.arange(1,len(pivot) + 1)
pivot['gap_to_leader'] = pivot['race_time'] - pivot['race_time'].iloc[0]
pivot.sort_values('prev_race_time', inplace=True)
pivot['prev_position'] = np.arange(1,len(pivot) + 1)
pivot['Gain/Loss'] = pivot.prev_position - pivot.position
pivot.sort_values('race_time', inplace=True)
pivot['interval'] = pivot.race_time.diff()
pivot = pivot.merge(df_dash[['Line','OEE  Start point', '⇗ OEE% progress', 'OEE% Target (2022)']], on='Line')
pivot

,Line,2021-04-15 00:00:00,2021-05-13 00:00:00,2021-06-10 00:00:00,2021-07-08 00:00:00,race_time,prev_race_time,Plant,position,gap_to_leader,prev_position,Gain/Loss,interval,OEE Start point,⇗ OEE% progress,OEE% Target (2022)
0,GAMMA1,89.156164,89.725767,89.926756,66.655259,335.463947,268.808688,SCOPPITO,1,0.000000,5,4,NaN,0.418683,0.085148,0.570
1,IMA C80/2,88.912853,89.656522,89.732693,73.920799,342.222868,268.302069,SCOPPITO,2,6.758921,4,2,6.758921,0.451031,0.043365,0.580
2,M22 Filling,88.956584,89.806345,90.057670,83.164274,351.984873,268.820600,Frankfurt,3,16.520927,6,3,9.762006,0.530068,0.120280,0.650
3,AL6,89.140448,90.323044,89.805598,82.931626,352.200716,269.269090,Frankfurt,4,16.736769,8,4,0.215843,0.332657,0.078541,0.450
4,M21 Filling,89.636132,91.584172,89.976645,83.183274,354.380223,271.196949,Frankfurt,5,18.916276,12,7,2.179507,0.599671,0.022006,0.650
5,M18 Filling,89.963066,90.596174,90.646232,83.703134,354.908607,271.205473,Frankfurt,6,19.444660,13,7,0.528385,0.443522,-0.010057,0.650
6,L18 Packaging Line,88.849900,89.426704,89.407996,88.364237,356.048837,267.684600,Tours,7,20.584890,1,-6,1.140230,0.377683,0.086173,0.547
7,TR200 Packaging Line,88.713009,89.591164,89.381944,88.704211,356.390327,267.686117,Lisieux,8,20.926380,2,-6,0.341490,0.483505,0.120926,0.650
8,SUPPO Packaging Line,88.587980,89.612821,89.992146,88.660102,356.853049,268.192947,Lisieux,9,21.389102,3,-6,0.462722,0.353021,-0.010676,0.530
9,L25 Packaging Line,88.852377,90.108842,90.011979,88.524036,357.497233,268.973197,Tours,10,22.033287,7,-3,0.644185,0.351564,0.001613,0.478


In [91]:
pivot.sort_values(pivot.columns[1], inplace=True)
pivot['apr_position'] = np.arange(1,len(pivot) + 1)
pivot.sort_values(pivot.columns[2], inplace=True)
pivot['may_position'] = np.arange(1,len(pivot) + 1)
pivot.sort_values(pivot.columns[3], inplace=True)
pivot['jun_position'] = np.arange(1,len(pivot) + 1)
pivot.sort_values(pivot.columns[4], inplace=True)
pivot['jly_position'] = np.arange(1,len(pivot) + 1)

pivot[pivot.columns[[0,-4,-3,-2,-1]]]

,Line,apr_position,may_position,jun_position,jly_position
0,GAMMA1,10,5,5,1
1,IMA C80/2,5,4,3,2
3,AL6,9,10,4,3
2,M22 Filling,7,7,9,4
4,M21 Filling,12,13,6,5
5,M18 Filling,13,12,13,6
11,AL5 Packaging 1,11,6,12,7
6,L18 Packaging Line,3,1,2,8
10,C9 Packaging Line,8,8,11,9
12,C2 Packaging Line,6,11,10,10


#### write this out for tableau

In [32]:
pivot.to_csv(output_dir + "leaderboard.csv")

In [33]:
df_weekly[['Week','Line','Changeover_Diff','Changeover_rolling_mean']][df_weekly['Line'].str.contains('AL6')]

,Week,Line,Changeover_Diff,Changeover_rolling_mean
3,W13-2021,AL6,-0.077980,0.184958
24,W14-2021,AL6,-0.083726,0.143095
34,W15-2021,AL6,-0.044684,0.114245
51,W16-2021,AL6,0.028627,0.106977
64,W17-2021,AL6,-0.012346,0.078945
72,W18-2021,AL6,0.027427,0.078701
78,W19-2021,AL6,-0.010930,0.086895
93,W20-2021,AL6,-0.046747,0.076246
113,W21-2021,AL6,0.065788,0.085130
118,W22-2021,AL6,-0.027045,0.080396


In [33]:
df_weekly[['Date','Plant','Line','lap_time']].groupby(['Line',pd.Grouper(key='Date', freq='W')]).sum().reset_index()

,Line,Date,lap_time
0,AL5 Packaging 1,2021-04-04,1.160776
1,AL5 Packaging 1,2021-04-11,0.355378
2,AL5 Packaging 1,2021-04-18,0.399490
3,AL5 Packaging 1,2021-04-25,0.183433
4,AL5 Packaging 1,2021-05-02,0.607727
...,...,...,...
141,TR200 Packaging Line,2021-05-16,0.392889
142,TR200 Packaging Line,2021-05-23,0.298976
143,TR200 Packaging Line,2021-05-30,0.527540
144,TR200 Packaging Line,2021-06-06,0.162539


In [34]:
df_weekly[['Plant','Line','lap_time']][df_weekly['Date'] == '2021-06-13']

,Plant,Line,lap_time
130,Frankfurt,M21 Filling,-4.816726
131,SUZANO,MEDISEAL PURAN,0.693566
132,Lisieux,IWK Packaging Line,0.693566
133,SUZANO,LINE 01 - UHLMANN 1880,0.693566
134,Tours,L18 Packaging Line,0.364237
135,Lisieux,TR200 Packaging Line,0.704211
136,Frankfurt,M18 Filling,-4.296866
137,Frankfurt,AL5 Packaging 1,0.294557
138,SCOPPITO,GAMMA1,-21.344741
139,Maisons-Alfort,C9 Packaging Line,0.404967
